In [1]:
%run training_functions.ipynb

/home/ac.cdavies/.vnv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-21 17:49:10.329156: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745257750.344653 4022096 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745257750.349105 4022096 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745257750.362046 4022096 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 

{'record': Value(dtype='string', id=None), 'text': Value(dtype='string', id=None)}
{'text': Value(dtype='string', id=None)}
{'text': Value(dtype='string', id=None)}
['train', 'validation', 'test']
['train', 'validation', 'test']
In, Out, get_ipython, exit, quit, open, accelerate, datasets, evaluate, math, np, os, peft, pickle, pytest, ipytest, pd, transformers, torch, time, threading, load_dataset, load_dataset_builder, get_dataset_split_names, get_dataset_config_names, LoftQConfig, LoraConfig, get_peft_model, PeftModelForCausalLM, AutoModelForCausalLM, AutoTokenizer, pipeline, TrainingArguments, Trainer, SFTTrainer, SFTConfig, @py_builtins, @pytest_ar, model_from_pkl, ds_builder1, ds_builder2, ds_builder3, ds_gs, ds_gst1_train, ds_gst1_test, ds_gst2_train, ds_gst2_test, search_with_strings, map_data, dp_tokenizer, print_trainable_parameters, metric, initialize_heartbeat, heartbeat, end_heartbeat, comp_metrics_output, compute_metrics, make_trainer, get_dataframe, get_training_output, 

In [2]:
import torch
import torch.nn as nn
from opacus.grad_sample import GradSampleModule
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist
import torch.nn as nn
import torch.optim as optim
import torch.multiprocessing as mp

In [3]:
lla_321, lla_321_tokenizer=model_from_pkl("Llama-3.2-1B-Instruct")

In [4]:
ds_gst1_train=load_dataset("LongSafari/open-genome", "stage1", split="train[:500]")
#print(ds_gst1[50])
ds_gst1_test=load_dataset("LongSafari/open-genome", "stage1", split="test[:50]")
print(get_dataset_split_names("LongSafari/open-genome", "stage1"))
ds_gst2_train=load_dataset("LongSafari/open-genome", "stage2", split="train[:500]")
ds_gst2_test=load_dataset("LongSafari/open-genome", "stage2", split="test[:50]")
print(get_dataset_split_names("LongSafari/open-genome", "stage2"))
# using a smaller dataset to make sure this works

['train', 'validation', 'test']
['train', 'validation', 'test']


In [5]:
#let's try to use a custom training loop
def preprocess_data(dataset):
    dataset=dataset.remove_columns(["text", "record"]) # pytorch does not accept this input
    dataset.set_format("torch") # ensures Tensors are returned
    return dataset


In [6]:
# ds_gst1_train=preprocess_data(ds_gst1_train)
# ds_gst1_test=preprocess_data(ds_gst1_test)

In [7]:
l_tokenized_stage1_train=map_data(ds_gst1_train, lla_321, lla_321_tokenizer)
l_tokenized_stage1_test=map_data(ds_gst1_test, lla_321, lla_321_tokenizer)

In [8]:
l_processed_train=preprocess_data(l_tokenized_stage1_train)
l_processed_test=preprocess_data(l_tokenized_stage1_test)

In [9]:
from torch.utils.data import DataLoader
train_dataloader=DataLoader(l_processed_train, shuffle=False, batch_size=1, pin_memory=True)
test_dataloader=DataLoader(l_processed_test, shuffle=False, batch_size=1, pin_memory=True)

In [10]:
# to fine-tune with LoRA, instantiate a base model (as above)
# create LoraConfig where LoRA-specific parameters are defined
config=LoraConfig(
    #inference_mode=False,
    r=8, #rank of update matrices, lower value results in smaller matrices with fewer parameters
    lora_alpha=16, #LoRA scaling factor
    #target_modules=["gate_proj", "down_proj", "up_proj", "q_proj", "v_proj", "k_proj", "o_proj"],
    task_type="CAUSAL_LM",
    lora_dropout=0.1, # dropout probability of LoRA layers
    bias="none", # specifies if bias parameters should be trained
    #modules_to_save=["decode_head"] #models apart from LoRA layers that are trainable
)

In [11]:
lla_lora_model=get_peft_model(lla_321, config)
print_trainable_parameters(lla_lora_model)

trainable params: 851968 || all params: 1236666368 || trainable%: 0.07


In [12]:
torch.cuda.device_count()

4

In [13]:
def setup(rank, world_size):
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355'
    # initialize the process group
    #size=dist.get_world_size()
    #rank=dist.get_rank()
    dist.init_process_group("gloo", rank=rank, world_size=world_size)

def cleanup():
    dist.destroy_process_group()

In [14]:
optimizer=torch.optim.AdamW(lla_lora_model.parameters(),
                           amsgrad=False, # the AMSGrad variant of this algorithm won't be used 
                            betas=(0.9, 0.999), # coefficients used for computing running averages of gradient and its square
                            capturable=False, # whether the instance will be captured in a CUDA graph
                            differentiable=False, # whether autogad should occur through the optimzer step in training
                            eps=1e-08, # added to denominator to improve numerical stablitity
                            foreach=None, # whether foreach implementation is used
                            fused=None, #whether the fused implementation is used
                            #initial_lr=2e-05,
                            lr=0.1, #learning rate
                            maximize=False, # whether the object is maximized with respect to params instead og
                            weight_decay=0.0)

In [15]:
l3_trainer=make_trainer(lla_lora_model, train_dataloader.dataset, test_dataloader.dataset, config,
                          SFTConfig(output_dir="test_trainer", eval_strategy="epoch",
                                    per_device_train_batch_size=1,
                                    max_grad_norm=1.0,
                                    num_train_epochs=1,
                                    logging_strategy="epoch",
                                    #logging_steps=6
                                    fp16=True,
                                   ), (optimizer))

In [16]:
from transformers import get_scheduler
from accelerate.test_utils.testing import get_backend
def dp_train(rank, world_size):
    dist.init_process_group("gloo", rank=rank, world_size=world_size)
    optimizer=l3_trainer.optimizer
    num_epochs=l3_trainer.args.num_train_epochs
    num_steps=num_epochs * len(train_dataloader)
    l3_trainer.lr_scheduler=get_scheduler(
                name="linear",
                optimizer=trainer.optimizer,
                num_warmup_steps=0,
                num_training_steps=num_steps,
            ) #this is the default learning rate scheduler from the trainer
    model=l3_trainer.model
    # if torch.cuda.device_count() > 1:
    #   print("Let's use", torch.cuda.device_count(), "GPUs!")
    #   # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    #   model = nn.DataParallel(lla_321)

    model.to(device)
    scaler = torch.cuda.amp.GradScaler() # used for mixed precision training
    
    #device, _, _ = get_backend() # automatically detects the underlying device type (CUDA, CPU, XPU, MPS, etc.)
    model.half()
    #model.to(device)
    model.train()
    print(f"{num_steps=}")
    for epoch in range(int(num_epochs)):
        for batch in train_dataloader:
            optimizer.zero_grad()
            with torch.autocast(device_type='cuda', dtype=torch.float16):
                batch = {k: v.to('cuda') for k, v in batch.items()}
                outputs = model(**batch)
    #         #print(model)
            
                loss = outputs.loss

            # scaling loss, 
            #backward is called on scaled loss to get scaled gradients
            scaler.scale(loss).backward()
            # gradients are unscaled before the optimizer is called
            scaler.step(trainer.optimizer)
            trainer.lr_scheduler.step()
            scaler.update()

            # # add if statement for steps/multiple epochs here
            # eval_loss, eval_accuracy = trainer.evaluate()
            # print(eval_loss)
            # Clear cache if necessary
            torch.cuda.empty_cache()
    

In [17]:
print(torch.cuda.device_count())

4


In [18]:
def main():
    world_size=torch.cuda.device_count()
    mp.spawn(dp_train,
             args=(world_size,),
             nprocs=world_size,
             join=True)

In [21]:
!torchrun --nnodes=1 --nproc_per_node=4 dp_training.py 

W0421 17:51:32.127000 4038785 torch/distributed/run.py:792] 
W0421 17:51:32.127000 4038785 torch/distributed/run.py:792] *****************************************
W0421 17:51:32.127000 4038785 torch/distributed/run.py:792] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0421 17:51:32.127000 4038785 torch/distributed/run.py:792] *****************************************
2025-04-21 17:51:36.657660: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745257896.672967 4039004 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-21 17:51:36.673309: E external/local_xla/xla/stream_executor/cuda/cuda_f

In [20]:
help(inner_model)

NameError: name 'inner_model' is not defined

In [ ]:
#l3_trainer.train()

In [ ]:
l3_trainer.state.log_history[0]['loss']

In [ ]:
l3_trainer.train_dataset

In [ ]:
dir(l3_trainer.prediction_step)

In [ ]:
# the Trainer object will call the optimizer's train() function at each training step if method is callable/exists
# maybe create a train method for your optimizer with the required steps for DPOptimizer 
def optimize(trainer):
    #for i in range(l3_trainer.args.num_train_epochs): might not be needed if the trainer executes this at steps/epochs
    losses=[] # empty loss list
    data_loader=trainer.get_train_dataloader
    optimizer=trainer.optimizer
    optimizer.zero_grad() # resets the gradients of the parameter tensors to zero

    outputs= trainer.#training output at this step/epoch
    loss=outputs[0]['loss'] #appropriate position in outputs
    loss.backward() # calculates gradient by backpropagating error from the current loss
    losses.append(loss.item()) #add the gradient value to the list
    optimizer.step() # perform optimization step to update the parameter
    
    

In [ ]:
optimizer.step

In [ ]:
optimizer.pre_step

In [ ]:
optimizer.train=optimize

In [ ]:
from opacus import PrivacyEngine
privacy_engine = PrivacyEngine(secure_mode=False)
model, optimizer, train_loader = privacy_engine.make_private(
        module=lla_lora_model,
        optimizer=optimizer,
        data_loader=train_dataloader,
        noise_multiplier=1.3,
        max_grad_norm=1.0,
        )

In [ ]:
lla_lora_model=get_peft_model(lla_321, config)

In [ ]:
l6_trainer=make_trainer(lla_lora_model, train_dataloader.dataset, test_dataloader.dataset, config,
                          SFTConfig(output_dir="test_trainer", eval_strategy="epoch",
                                    per_device_train_batch_size=1,
                                    max_grad_norm=1.0,
                                    num_train_epochs=1,
                                    logging_strategy="epoch",
                                    #logging_steps=6,
                                    ), optimizer)

In [ ]:
l6_trainer.train()

In [ ]:
dir(optimizer)

In [ ]:
l6_trainer.optimizer.train(l5_trainer)

In [ ]:
l5_trainer.train()

In [ ]:
dir(l3_trainer.optimizer)

In [ ]:
type(l3_trainer)

In [ ]:
# run the .get_optimizer_cls_and_kwargs on the trainer
l3_trainer.get_optimizer_cls_and_kwargs(SFTConfig(output_dir="test_trainer", eval_strategy="epoch",
                                    per_device_train_batch_size=1,
                                    max_grad_norm=1.0,
                                    num_train_epochs=1,
                                    logging_strategy="epoch",
                                    #logging_steps=6
                                   ))

In [ ]:
l4_trainer=make_trainer(lla_lora_model, train_dataloader.dataset, test_dataloader.dataset, config,
                          SFTConfig(output_dir="test_trainer", eval_strategy="epoch",
                                    per_device_train_batch_size=1,
                                    max_grad_norm=1.0,
                                    num_train_epochs=1,
                                    logging_strategy="epoch",
                                    #logging_steps=6
                                   ))

In [ ]:
l4_trainer.train()

In [ ]:
dir(l4_trainer.optimizer)
# it seems that the grad_samples parameter doesn't exist in a regular AdamW optimizer, it is just for the DPOptimizer version
# when the .get_optimizer_cls_and_kwargs function is ran on the trainer with the DPOptimizer, it retrieves arguments from the normal AdamW optimizer

In [ ]:
l4_trainer.get_optimizer_cls_and_kwargs(SFTConfig(output_dir="test_trainer", eval_strategy="epoch",
                                    per_device_train_batch_size=1,
                                    max_grad_norm=1.0,
                                    num_train_epochs=1,
                                    logging_strategy="epoch",
                                    #logging_steps=6
                                   ))
# for the default and imported optimizers, the same arguments are retrieved by this function